---

<a href="https://github.com/rraadd88/roux/blob/master/examples/roux_lib_df.ipynb"><img align="right" style="float:right;" src="https://img.shields.io/badge/-source-cccccc?style=flat-square"></a>

## ⌗ Dataframes.

**Optional requirements**

In [1]:
# to show logging messages
import logging
logging.getLogger().setLevel(logging.INFO)

## Import `r`oux-`d`ataframe attributes

In [2]:
import roux.lib.dfs as rd

## Basic data checks 

**Demo data**

In [3]:
import seaborn as sns
import numpy as np
data=sns.load_dataset('iris')
data=(
    data
    .assign(
    **{
        ## insert missing values 
        "sepal_length with missing values":lambda df: df['sepal_length'].apply(lambda x: np.nan if x>5 else x),
    }
    )
)
data.head(1)

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_length with missing values
0,5.1,3.5,1.4,0.2,setosa,NaN


### Missing values 

Check the counts

In [4]:
data.rd.check_na(subset=['sepal_length with missing values'],perc=True) # returns counts

sepal_length with missing values    78.666667
dtype: float64

.. in chained operations

In [5]:
(data
    .rd.check_na(subset=['sepal_length with missing values'],out=False) # returns the original dataframe
    .head(1) 
)

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_length with missing values
0,5.1,3.5,1.4,0.2,setosa,NaN


### Duplicates

Check duplicate rows

In [6]:
data.rd.check_dups(subset='sepal_length').head() # returns duplicate rows

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_length with missing values
38,4.4,3.0,1.3,0.2,setosa,4.4
42,4.4,3.2,1.3,0.2,setosa,4.4
8,4.4,2.9,1.4,0.2,setosa,4.4
3,4.6,3.1,1.5,0.2,setosa,4.6
47,4.6,3.2,1.4,0.2,setosa,4.6


Check counts in chained operations

In [7]:
(data
    .rd.check_dups(subset='sepal_length',out=False) # returns the original dataframe
    .head()
)

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_length with missing values
0,5.1,3.5,1.4,0.2,setosa,NaN
1,4.9,3.0,1.4,0.2,setosa,4.9
2,4.7,3.2,1.3,0.2,setosa,4.7
3,4.6,3.1,1.5,0.2,setosa,4.6
4,5.0,3.6,1.4,0.2,setosa,5.0


In [8]:
# for testing
assert data.shape==(150, 6), data.shape

### Unique values

Check counts

In [9]:
data.rd.check_nunique(subset=['species']) # returns counts

species    3
dtype: int64

In [10]:
data.rd.check_nunique(subset='sepal_length', groupby='species') # .. grouped by a column/s

species
setosa        15
versicolor    21
virginica     21
Name: sepal_length, dtype: int64

.. in chained operations

In [11]:
(data
    .rd.check_nunique(subset='species',out=False).head(1) # returns the original dataframe
    .head(1)
)

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_length with missing values
0,5.1,3.5,1.4,0.2,setosa,NaN


## Validate

### In chained operations

In [12]:
_=(data
   .drop_duplicates() # preparing demo data
   
   # data validations
   .rd.assert_no_na(subset=['sepal_length']) # validate no missing values
   .rd.assert_no_dups() # validate no duplicates
  )

In [13]:
_=(data
   .drop_duplicates().drop(['sepal_length with missing values'],axis=1) # preparing demo data
   
   # data validations
   .rd.assert_dense() # validate both no missing values and no duplicates
  )

## Logging
### Changes in the dataframe shapes

In [14]:
_=data.log.drop_duplicates()

.. in chained operations

In [15]:
_=(data
   .log.drop_duplicates() # logs the change in the shape of the dataframe (e.g. length)
   .log('sepal_length') # counts of unique values
   .log('sepal_length',groupby='species') # .. grouped by a column/s
  )

## Filter 
### Using a dictionary

In [16]:
out=data.rd.filter_rows({'species':'setosa'})
out.head(1)

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_length with missing values
0,5.1,3.5,1.4,0.2,setosa,NaN


In [17]:
# for testing
assert out.shape==(50, 6), out.shape

## Groupby

### Sample

In [18]:
data.rd.groupby_sample('species').shape ## sample a group

(50, 6)

### Aggregate nested groups

In [19]:
## demo data
data2=(data
    .melt(id_vars=['species'],value_vars=["sepal_length","sepal_width","petal_length","petal_width"],var_name='part measurement')
    .assign(
        part=lambda df: df['part measurement'].str.split('_',expand=True)[0],
        measurement=lambda df: df['part measurement'].str.split('_',expand=True)[1],
    )
    )
data2.head(1)

,species,part measurement,value,part,measurement
0,setosa,sepal_length,5.1,sepal,length


In [20]:
## aggregate
data2.rd.groupby_agg_nested(
    groupby=['species'], # groupby columns
    subset=["part measurement","part","measurement"], ## columns with nested groups
    func={"value": np.mean}, ## column with values and agg function to apply
    )

,species,value
0,setosa,2.5355
1,versicolor,3.5730
2,virginica,4.2850


## Merge

**Demo data**

In [21]:
data2=data.groupby('species').head(1)

### Validation of changes in table shapes

In [22]:
out=data.log.merge( # logs the changes in the shapes of the dataframe
    right=data2,
    how='inner',
    on='species',
    validate='m:1',
    validate_equal_length=True,
    # validate_no_decrease_length=True,
    )
out.head(1)

,sepal_length_x,sepal_width_x,petal_length_x,petal_width_x,species,sepal_length with missing values_x,sepal_length_y,sepal_width_y,petal_length_y,petal_width_y,sepal_length with missing values_y
0,5.1,3.5,1.4,0.2,setosa,NaN,5.1,3.5,1.4,0.2,NaN


In [23]:
# for testing
assert out.shape==(150, 11), out.shape

## Melt

**Demo data with paired IDs and values**

In [24]:
np.random.seed(1)
data=(
    data
    .assign(
    **{
        ## create paired ids
        "id 1":lambda df: range(len(df)),
        "id 2":lambda df: range(len(df))[::-1],
    }
    )
    .rename(
        columns={
            "sepal_length": "value 1",
            "sepal_width": "value 2",
            
        },
        errors='raise',
    )
    .loc[:,['id 1','id 2','value 1','value 2']]
).sample(5).reset_index(drop=True)
## example order
order=list(set(data['id 1'].tolist()+data['id 2'].tolist()))
order=list(np.random.choice(order,len(order),replace=False))
data

,id 1,id 2,value 1,value 2
0,14,135,5.8,4.0
1,98,51,5.1,2.5
2,75,74,6.6,3.0
3,16,133,5.4,3.9
4,131,18,7.9,3.8


In [25]:
# for testing
assert data.shape==(5,4), data.shape

### Tables with paired IDs

In [26]:
out=data.rd.melt_paired(suffixes=['1','2'])
out

,suffix,id,value
0,1,14,5.8
1,1,98,5.1
2,1,75,6.6
3,1,16,5.4
4,1,131,7.9
0,2,135,4.0
1,2,51,2.5
2,2,74,3.0
3,2,133,3.9
4,2,18,3.8


In [27]:
# for testing
assert out.shape==(10,3), out.shape

## Sort 

### Paired columns by values

In [28]:
out=data.rd.sort_columns_by_values(['id 1','id 2'])
out

,equal,sorted,id 1,id 2,value 1,value 2
0,False,False,14,135,5.8,4.0
3,False,False,16,133,5.4,3.9
1,False,True,51,98,2.5,5.1
2,False,True,74,75,3.0,6.6
4,False,True,18,131,3.8,7.9


In [29]:
# for testing
assert out['id 1'].tolist()==[14, 16, 51, 74, 18], out['id 1'].tolist()

### Paired columns by order

In [30]:
order

[133, 135, 14, 51, 131, 18, 75, 16, 98, 74]

In [31]:
out=data.rd.sort_columns_by_values(
    ['id 1','id 2'],
    order=order, # order of the ids
    clean=True,
)
out

,id 1,id 2,value 1,value 2
2,75,74,6.6,3.0
4,131,18,7.9,3.8
0,135,14,4.0,5.8
1,51,98,2.5,5.1
3,133,16,3.9,5.4


In [32]:
# for testing
assert out['id 1'].tolist()==[75, 131, 135, 51, 133], out['id 1'].tolist()

## Mapping between columns

**Demo data**

In [33]:
data=(
    data
    .loc[:,['id 1']].head(10)
    .assign(
    **{
        ## create shuffled ids
        "id 2":lambda df: np.random.choice(df['id 1'],len(df)),
    }
    )
)
data

,id 1,id 2
0,14,16
1,98,98
2,75,98
3,16,131
4,131,131


### Check counts

In [34]:
out=data.rd.check_mappings(subset=['id 1','id 2'])
out

,,,mappings count
mapping,id 1 count,id 2 count,
1:1,1,1,1
m:1,2,1,4


In [35]:
# for testing
assert out['mappings count'].tolist()==[1, 4], out['mappings count'].tolist()

### Classify

In [36]:
out=data.rd.classify_mappings(subset=['id 1','id 2'])
out

,id 1,id 2,id 1 count,id 2 count,mapping
0,14,16,1,1,1:1
1,98,98,2,1,m:1
2,75,98,2,1,m:1
3,16,131,2,1,m:1
4,131,131,2,1,m:1


In [37]:
# for testing
assert out['mapping'].tolist()==['1:1', 'm:1', 'm:1', 'm:1', 'm:1'], out['mapping'].tolist()

### Filter

In [38]:
out=data.rd.get_mappings(
    subset=['id 1','id 2'],
    keep="2:1", # keep 3:1 mappings
    clean=False,
)
out

,id 1,id 2,id 1 count,id 2 count,mapping
1,98,98,2,1,m:1
2,75,98,2,1,m:1
3,16,131,2,1,m:1
4,131,131,2,1,m:1


In [39]:
# for testing
assert out.shape==(4,5), out

In [40]:
out=data.rd.get_mappings(
    subset=['id 1','id 2'],
    keep='1:1', ## keep 1:1 mappings
    clean=True,
    )
out

,id 1,id 2,mapping
0,14,16,1:1


In [41]:
# for testing
assert out['mapping'].tolist()==['1:1'], out['mapping'].tolist()

### Validate

In [42]:
data.rd.classify_mappings(subset=['id 1','id 2'])

,id 1,id 2,id 1 count,id 2 count,mapping
0,14,16,1,1,1:1
1,98,98,2,1,m:1
2,75,98,2,1,m:1
3,16,131,2,1,m:1
4,131,131,2,1,m:1


In [43]:
# for testing
assert len(data)==5

In [44]:
data.head(1).rd.assert_1_1_mappings(subset=['id 1','id 2'])

## Documentation
[`roux.lib.df`](https://github.com/rraadd88/roux#module-rouxlibdf)